In [ ]:
import json 
import pandas as pd
import numpy as np
with open('natural_question.json') as f: 
    data = json.load(f)

In [ ]:
import tensorflow.compat.v1 as tf
import os 
import shutil
import csv
import pandas as pd
import IPython

tf.get_logger().setLevel('ERROR')

from tapas.utils import tf_example_utils
from tapas.protos import interaction_pb2
from tapas.utils import number_annotation_utils
from tapas.scripts import prediction_utils

print('done')

done


In [ ]:
os.makedirs('results/sqa/tf_examples', exist_ok=True)
os.makedirs('results/sqa/model', exist_ok=True)
with open('results/sqa/model/checkpoint', 'w') as f:
  f.write('model_checkpoint_path: "model.ckpt-0"')
for suffix in ['.data-00000-of-00001', '.index', '.meta']:
  shutil.copyfile(f'tapas_sqa_base/model.ckpt{suffix}', f'results/sqa/model/model.ckpt-0{suffix}')

In [ ]:
max_seq_length = 512
vocab_file = "tapas_sqa_base/vocab.txt"
config = tf_example_utils.ClassifierConversionConfig(
    vocab_file=vocab_file,
    max_seq_length=max_seq_length,
    max_column_id=max_seq_length,
    max_row_id=max_seq_length,
    strip_column_names=False,
    add_aggregation_candidates=False,
)
converter = tf_example_utils.ToClassifierTensorflowExample(config)

In [ ]:
def convert_interactions_to_examples(tables_and_queries):
  """Calls Tapas converter to convert interaction to example."""
  for idx, (table, queries) in enumerate(tables_and_queries):
    interaction = interaction_pb2.Interaction()
    for position, query in enumerate(queries):
      question = interaction.questions.add()
      question.original_text = query
      question.id = f"{idx}-0_{position}"
    for header in table[0]:
      interaction.table.columns.add().text = header
    for line in table[1:]:
      row = interaction.table.rows.add()
      for cell in line:
        row.cells.add().text = cell
    number_annotation_utils.add_numeric_values(interaction)
    for i in range(len(interaction.questions)):
      try:
        yield converter.convert(interaction, i)
      except ValueError as e:
        print(f"Can't convert interaction: {interaction.id} error: {e}")
        
def write_tf_example(filename, examples):
  with tf.io.TFRecordWriter(filename) as writer:
    for example in examples:
      writer.write(example.SerializeToString())

def predict(table_data, queries):
  table = table_data
  examples = convert_interactions_to_examples([(table, queries)])
  write_tf_example("results/sqa/tf_examples/test.tfrecord", examples)
  write_tf_example("results/sqa/tf_examples/random-split-1-dev.tfrecord", [])
  
  ! python -m tapas.run_task_main \
    --task="SQA" \
    --output_dir="results" \
    --noloop_predict \
    --test_batch_size={len(queries)} \
    --tapas_verbosity="ERROR" \
    --compression_type= \
    --init_checkpoint="tapas_sqa_base/model.ckpt" \
    --bert_config_file="tapas_sqa_base/bert_config.json" \
    --mode="predict" 2> error


  results_path = "results/sqa/model/test_sequence.tsv"
  all_coordinates = []
  df = pd.DataFrame(table[1:], columns=table[0])
  display(IPython.display.HTML(df.to_html(index=False)))
  print()
  with open(results_path) as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    for row in reader:
      coordinates = prediction_utils.parse_coordinates(row["answer_coordinates"])
      all_coordinates.append(coordinates)
      answers = ', '.join([table[row + 1][col] for row, col in coordinates])
      position = int(row['position'])
      print(">", queries[position])
      print(answers)
  return all_coordinates

In [ ]:
df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vS8SzaERcKJOD_EzrtCDK1dX1zkoMochlA9iHoHg_RSw3V8bkpfk1mpw4pfL5RdtSOyx_oScsUtyXyk/pub?gid=43720681&amp;single=true&amp&output=csv")
df = df.query('State == "OH"')
df = df.iloc[10:19]
df

,Date,State,Cases_Total,Cases_White,Cases_Black,Cases_Latinx,Cases_Asian,Cases_AIAN,Cases_NHPI,Cases_Multiracial,...,Tests_Latinx,Tests_Asian,Tests_AIAN,Tests_NHPI,Tests_Multiracial,Tests_Other,Tests_Unknown,Tests_Ethnicity_Hispanic,Tests_Ethnicity_NonHispanic,Tests_Ethnicity_Unknown
598,20210131,OH,895792.0,520420.0,93394.0,NaN,12378.0,983.0,646.0,21480.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
654,20210127,OH,878284.0,509117.0,91733.0,NaN,12142.0,961.0,641.0,21106.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
710,20210124,OH,864322.0,499074.0,90208.0,NaN,11975.0,949.0,627.0,20765.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
766,20210120,OH,842433.0,485676.0,88139.0,NaN,11658.0,931.0,608.0,20329.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
822,20210117,OH,826754.0,474468.0,86298.0,NaN,11363.0,908.0,599.0,19930.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
878,20210113,OH,799639.0,458196.0,83865.0,NaN,11089.0,886.0,587.0,19402.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
934,20210110,OH,777065.0,443518.0,81661.0,NaN,10808.0,863.0,574.0,18934.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
990,20210106,OH,742817.0,423353.0,78994.0,NaN,10463.0,844.0,557.0,18189.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1046,20210103,OH,721481.0,409049.0,40740.0,NaN,10222.0,812.0,541.0,17724.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
count = 0 
count2 = 0
question_output = {}
for i in range(len(data)): 
    if data[str(i+1)][0]['question_template_id'] == '2q8':
        q = data[str(i+1)][0]['question']
        ent = data[str(i+1)][0]['entities']
        if ent[0] == 'new cases': 
            continue
        if count%3 == 1 or count%3 == 2: 
            count += 1
            continue
        print(count2)
        print(q)
        print(ent)
        if ent[2] == 'African-American' or ent[2] == 'Black': 
            if ent[0] == 'confirmed cases': 
                df2 = df[['Date','State', 'Cases_Black']]
            else:
                df2 = df[['Date', 'State', 'Deaths_Black']]
        elif ent[2] == 'White' or ent[2] == 'Caucasian':
            if ent[0] == 'confirmed cases': 
                df2 = df[['Date', 'State', 'Cases_White']]
            else: 
                df2 = df[['Date', 'State', 'Deaths_White']]
        elif ent[2] == 'Asian': 
            if ent[0] == 'confirmed cases': 
                df2 = df[['Date', 'State', 'Cases_Asian']]
            else: 
                df2 = df[['Date', 'State', 'Deaths_Asian']]
        elif ent[2] == 'Hispanic' or ent[2] == 'Latino:':
            if ent[0] == 'confirmed cases': 
                df2 = df[['Date', 'State', 'Cases_Latinx']]
            else: 
                df2 = df[['Date', 'State', 'Deaths_Latinx']]
        elif ent[2] == 'American Indian' or ent[2] == 'Alaska Native' or ent[2] == 'American Indian or Alaska Native': 
            if ent[0] == 'confirmed cases': 
                df2 = df[['Date', 'State', 'Cases_AIAN']]
            else: 
                df2 = df[['Date', 'State', 'Deaths_AIAN']]
        elif ent[2] == 'Native Hawaiian' or ent[2] == 'Pacific Islander' or ent[2] == 'Pacific Islander and Native Hawaiian': 
            if ent[0] == 'confirmed cases': 
                df2 = df[['Date', 'State', 'Cases_NHPI']]
            else: 
                df2 = df[['Date', 'State', 'Deaths_NHPI']]
        else: 
            if ent[0] == 'confirmed cases': 
                df2 = df[['Date', 'State', 'Cases_Multiracial']]
            else: 
                df2 = df[['Date', 'State', 'Deaths_Multiracial']]
        df2 = df2.astype(str)
        list_of_list = [[]]
        list_of_list[0] = list(df2.columns)
        list_of_list.extend(df2.values.tolist())
        list_of_list
        result = predict(list_of_list, [q])
        res = ""
        count1 = 0
        print(result)
        for i in result[0]: 
            if count1 == 0:
                count1 +=1 
                res += list_of_list[i[0]+1][i[1]]
            else: 
                res += ', ' + list_of_list[i[0]+1][i[1]]
            question_output[q] = res
        count +=1
        count2 +=1
print(count)
print(count2)
question_output

0
How many African-American confirmed cases occurred in Ohio in 1st, January?
['confirmed cases', 'Ohio', 'African-American', 'January']
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


Date,State,Cases_Black
20210131,OH,93394.0
20210127,OH,91733.0
20210124,OH,90208.0
20210120,OH,88139.0
20210117,OH,86298.0
20210113,OH,83865.0
20210110,OH,81661.0
20210106,OH,78994.0
20210103,OH,40740.0



> How many African-American confirmed cases occurred in Ohio in 1st, January?
91733.0, 88139.0, 40740.0, 81661.0, 90208.0, 86298.0, 83865.0, 93394.0, 78994.0
[{(1, 2), (3, 2), (8, 2), (6, 2), (2, 2), (4, 2), (5, 2), (0, 2), (7, 2)}]
1
How many African-American confirmed cases occurred in Ohio in 4th, January?
['confirmed cases', 'Ohio', 'African-American', 'January']
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


Date,State,Cases_Black
20210131,OH,93394.0
20210127,OH,91733.0
20210124,OH,90208.0
20210120,OH,88139.0
20210117,OH,86298.0
20210113,OH,83865.0
20210110,OH,81661.0
20210106,OH,78994.0
20210103,OH,40740.0



> How many African-American confirmed cases occurred in Ohio in 4th, January?
91733.0, 88139.0, 40740.0, 81661.0, 90208.0, 86298.0, 83865.0, 78994.0
[{(1, 2), (3, 2), (8, 2), (6, 2), (2, 2), (4, 2), (5, 2), (7, 2)}]
2
How many African-American confirmed cases occurred in Ohio in 7th, January?
['confirmed cases', 'Ohio', 'African-American', 'January']
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


...

Date,State,Deaths_Multiracial
20210131,OH,218.0
20210127,OH,214.0
20210124,OH,210.0
20210120,OH,206.0
20210117,OH,198.0
20210113,OH,191.0
20210110,OH,184.0
20210106,OH,182.0
20210103,OH,166.0



> How many mixed deaths occurred in Ohio in 23rd, January?
182.0
[{(7, 2)}]
279
How many mixed deaths occurred in Ohio in 26th, January?
['deaths', 'Ohio', 'mixed', 'January']
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


Date,State,Deaths_Multiracial
20210131,OH,218.0
20210127,OH,214.0
20210124,OH,210.0
20210120,OH,206.0
20210117,OH,198.0
20210113,OH,191.0
20210110,OH,184.0
20210106,OH,182.0
20210103,OH,166.0



> How many mixed deaths occurred in Ohio in 26th, January?
182.0
[{(7, 2)}]
840
280


{'How many African-American confirmed cases occurred in Ohio in 1st, January?': '91733.0, 88139.0, 40740.0, 81661.0, 90208.0, 86298.0, 83865.0, 93394.0, 78994.0',
 'How many African-American confirmed cases occurred in Ohio in 4th, January?': '91733.0, 88139.0, 40740.0, 81661.0, 90208.0, 86298.0, 83865.0, 78994.0',
 'How many African-American confirmed cases occurred in Ohio in 7th, January?': '91733.0, 88139.0, 40740.0, 81661.0, 90208.0, 86298.0, 83865.0, 78994.0',
 'How many African-American confirmed cases occurred in Ohio in 10th, January?': '91733.0, 88139.0, 40740.0, 81661.0, 90208.0, 86298.0, 83865.0, 78994.0',
 'How many African-American confirmed cases occurred in Ohio in 13th, January?': '91733.0, 88139.0, 40740.0, 81661.0, 90208.0, 86298.0, 83865.0, 78994.0',
 'How many African-American confirmed cases occurred in Ohio in 16th, January?': '91733.0, 88139.0, 40740.0, 81661.0, 90208.0, 86298.0, 83865.0, 78994.0',
 'How many African-American confirmed cases occurred in Ohio in 

In [ ]:
count = 0
count2 = 0
date_list = ['in 3rd, January', 'in 6th, January', 'in 10th, January', 'in 13th, January', 'in 17th, January', 'in 20th, January', 'in 24th, January', 'in 27th, January', 'in 31st, January']

for i in range(len(data)): 
    if data[str(i+1)][0]['question_template_id'] == '2q8':
        q = data[str(i+1)][0]['question']
        ent = data[str(i+1)][0]['entities']
        if ent[0] == 'new cases': 
            continue
        cont_check = False
        for j in date_list: 
            if j in q:
                cont_check = True
        if cont_check == False:
            continue
        if count%3 == 1 or count%3 == 2: 
            count += 1
            continue
        print(count2)
        print(q)
        print(ent)
        count +=1
        count2 +=1
print(count2)
print(count)

0
How many African-American confirmed cases occurred in Ohio in 3rd, January?
['confirmed cases', 'Ohio', 'African-American', 'January']
1
How many African-American confirmed cases occurred in Ohio in 13th, January?
['confirmed cases', 'Ohio', 'African-American', 'January']
2
How many African-American confirmed cases occurred in Ohio in 24th, January?
['confirmed cases', 'Ohio', 'African-American', 'January']
3
How many Hispanic confirmed cases occurred in Ohio in 6th, January?
['confirmed cases', 'Ohio', 'Hispanic', 'January']
4
How many Hispanic confirmed cases occurred in Ohio in 17th, January?
['confirmed cases', 'Ohio', 'Hispanic', 'January']
5
How many Hispanic confirmed cases occurred in Ohio in 27th, January?
['confirmed cases', 'Ohio', 'Hispanic', 'January']
6
How many Latino confirmed cases occurred in Ohio in 10th, January?
['confirmed cases', 'Ohio', 'Latino', 'January']
7
How many Latino confirmed cases occurred in Ohio in 20th, January?
['confirmed cases', 'Ohio', 'Latino

In [ ]:
len(question_output)


267

In [ ]:
with open('2q8data.json', 'w') as outfile:
    json.dump(question_output, outfile)